In [3]:
!pip install gradio pandas --quiet

In [4]:
import gradio as gr
import pandas as pd
from datetime import datetime
import os

# Daftar gejala
gejala_list = [
    "Tidak bisa menyala",
    "Lampu indikator menyala tapi layar gelap",
    "Layar berkedip-kedip",
    "Suara kipas terdengar keras",
    "Laptop mati tiba-tiba",
    "Baterai cepat habis",
    "Muncul pesan overheating"
]

# Aturan pakar
rules = [
    {
        "gejala": {"Tidak bisa menyala", "Lampu indikator menyala tapi layar gelap"},
        "diagnosa": "Kemungkinan kerusakan VGA",
        "confidence": 0.8
    },
    {
        "gejala": {"Laptop mati tiba-tiba", "Suara kipas terdengar keras"},
        "diagnosa": "Kemungkinan Overheating",
        "confidence": 0.9
    },
    {
        "gejala": {"Baterai cepat habis", "Muncul pesan overheating"},
        "diagnosa": "Kemungkinan Baterai Rusak",
        "confidence": 0.7
    },
    {
        "gejala": {"Layar berkedip-kedip"},
        "diagnosa": "Kemungkinan RAM atau VGA bermasalah",
        "confidence": 0.6
    }
]

# Fungsi sistem pakar
def sistem_pakar(input_gejala):
    hasil = []
    input_set = set(input_gejala)

    for rule in rules:
        if rule["gejala"].issubset(input_set):
            hasil.append(f"- {rule['diagnosa']} (Keyakinan: {int(rule['confidence']*100)}%)")

    if not hasil:
        hasil_text = "Belum bisa menyimpulkan kerusakan berdasarkan gejala tersebut."
    else:
        hasil_text = "\n".join(hasil)

    # Simpan riwayat ke CSV
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data = {
        "waktu": waktu,
        "gejala_dipilih": "; ".join(input_gejala),
        "hasil_diagnosa": hasil_text
    }
    df = pd.DataFrame([data])

    if os.path.exists("riwayat_diagnosa.csv"):
        df.to_csv("riwayat_diagnosa.csv", mode='a', header=False, index=False)
    else:
        df.to_csv("riwayat_diagnosa.csv", index=False)

    return hasil_text

# UI Gradio
iface = gr.Interface(
    fn=sistem_pakar,
    inputs=gr.CheckboxGroup(choices=gejala_list, label="Pilih Gejala Kerusakan Laptop"),
    outputs=gr.Textbox(label="Hasil Diagnosa"),
    title="💻 Sistem Pakar Diagnosa Kerusakan Laptop",
    description="Sistem akan memberikan kemungkinan diagnosa berdasarkan gejala yang dipilih. Riwayat disimpan ke file CSV."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://6dbe6ad5518bf9b3a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
